In [30]:
import json

import pandas as pd
import numpy as np

import re



In [23]:
raw_data_str = open("../AllCards.json").read()

In [17]:
raw_data_str = "[" + raw_data_str + "]"

In [24]:
raw_data_json = json.loads(raw_data_str)

In [68]:
i = 0
for card in raw_data_json:
    print(raw_data_json[card])
    i += 1
    if i > 10:
        break

{'colorIdentity': ['G', 'R'], 'colors': ['G', 'R'], 'convertedManaCost': 6.0, 'foreignData': [], 'isReserved': False, 'layout': 'normal', 'legalities': {}, 'manaCost': '{2}{R}{R}{G}{G}', 'name': '"Ach! Hans, Run!"', 'printings': ['UNH'], 'rulings': [], 'subtypes': [], 'supertypes': [], 'text': 'At the beginning of your upkeep, you may say "Ach Hans, run It\'s the . . ." and the name of a creature card. If you do, search your library for a card with that name, put it onto the battlefield, then shuffle your library. That creature gains haste. Exile it at the beginning of the next end step.', 'type': 'Enchantment', 'types': ['Enchantment'], 'uuid': '84f2c8f5-8e11-4639-b7de-00e4a2cbabee'}
{'colorIdentity': ['B'], 'colors': ['B'], 'convertedManaCost': 3.0, 'foreignData': [], 'isReserved': False, 'layout': 'normal', 'legalities': {}, 'manaCost': '{2}{B}', 'name': '"Rumors of My Death . . ."', 'printings': ['UST'], 'rulings': [], 'subtypes': [], 'supertypes': [], 'text': '{3}{B}, Exile a perm

In [26]:
raw_data_json["AWOL"]

{'colorIdentity': ['W'],
 'colors': ['W'],
 'convertedManaCost': 3.0,
 'foreignData': [],
 'isReserved': False,
 'layout': 'normal',
 'legalities': {},
 'manaCost': '{2}{W}',
 'name': 'AWOL',
 'printings': ['UNH'],
 'rulings': [],
 'subtypes': [],
 'supertypes': [],
 'text': 'Exile target attacking creature. Then remove it from the game. Then put it into the absolutely-removed-from-the-freaking-game-forever zone.',
 'type': 'Instant',
 'types': ['Instant'],
 'uuid': 'fcf3447a-ec7d-47d7-bf6b-e2bbe0ac58df'}

In [44]:
raw_data_json["AWOL"].get("type", 0)

'Instant'

In [27]:
len(raw_data_json.keys())

19267

In [60]:
creatures_ls = [card for card in raw_data_json if "Creature" in raw_data_json[card].get("type", 0)]

In [61]:
len(creatures_ls)

10062

In [145]:
# Format for feature vectors will be power, toughness, flying, firststrike, doublestrike, trample, 
# vigilance, defender, haste, menace, fear, deathtouch, indestructible, hexproof/shroud
columns = ["cid_c", "cid_w", "cid_u", "cid_g", "cid_b", "cid_r", "mc_c", "mc_w", "mc_u", "mc_b", "mc_r", "mc_g",\
           "power", "toughness", "flying", \
           "firststrike", "doublestrike", "trample", "vigilance", "defender", \
           "haste", "menace", "fear", "deathtouch", "indestructible", "hexproof/shroud"]

In [146]:
card_vectors_raw = np.zeros(shape = (len(creatures_ls), len(columns)))
card_vectors_raw

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [147]:
card_vectors_raw.shape

(10062, 26)

In [140]:
# test regex behavior re (har!) casting cost
test_cost = raw_data_json["Wrath of God"].get("manaCost", "")
test_cost_2 = '{2}{W}{W}{W}'

In [141]:
matches = re.findall("{W}", test_cost)

In [142]:
len(matches)

2

In [143]:
matches = re.search("{(\d*)}", test_cost_2)

In [144]:
int(matches.groups()[0])

2

In [156]:
def produce_vector(card):
    cid_c = int(len(card.get("colorIdentity", [])) == 0)
    cid_w = int('W' in card.get("colorIdentity", []))
    cid_u = int('U' in card.get("colorIdentity", []))
    cid_b = int('B' in card.get("colorIdentity", []))
    cid_r = int('R' in card.get("colorIdentity", []))
    cid_g = int('G' in card.get("colorIdentity", []))
    flying = int("flying" in card.get("text", "").lower())
    trample = int("trample" in card.get("text", "").lower())
    double_strike = int("double strike" in card.get("text", "").lower())
    first_strike = int("first strike" in card.get("text", "").lower())
    vigilance = int("vigilance" in card.get("text", "").lower())
    haste = int("haste" in card.get("text", "").lower())
    menace = int("menace" in card.get("text", "").lower())
    fear = int("fear" in card.get("text", "").lower())
    shadow = int("shadow" in card.get("text", "").lower())
    flashback = int("flashback" in card.get("text", "").lower())
    buyback = int("buyback" in card.get("text", "").lower())
    defender = int("defender" in card.get("text", "").lower())
    deathtouch = int("deathtouch" in card.get("text", "").lower())
    indestructible = int("indestructible" in card.get("text", "").lower())
    hexproof = int("hexproof" in card.get("text", "").lower())
    shroud = int("shroud" in card.get("text", "").lower())

    power = int(card.get("power", "").lower())
    toughness = int(card.get("toughness", "").lower())

    mc_c = 0
    mc_c_result = re.search("{(\d*)}", card.get("manaCost", ""))
    if mc_c_result is not None:
        mc_c = int(matches.groups()[0])
    mc_w = len(re.findall("{W}", card.get("manaCost", "")))
    mc_u = len(re.findall("{U}", card.get("manaCost", "")))
    mc_b = len(re.findall("{B}", card.get("manaCost", "")))
    mc_r = len(re.findall("{R}", card.get("manaCost", "")))
    mc_g = len(re.findall("{G}", card.get("manaCost", "")))
    
    return [cid_c , cid_w , cid_u , cid_b ,cid_r , cid_g , mc_c, mc_w, mc_u, mc_b, mc_r, mc_g, \
            power, toughness, flying , trample , double_strike ,\
            first_strike , vigilance , haste , menace , fear , shadow , flashback , buyback ,\
            defender , deathtouch , indestructible , hexproof , shroud ]

In [158]:
list(zip(columns, produce_vector(raw_data_json["Craw Wurm"])))

[('cid_c', 0),
 ('cid_w', 0),
 ('cid_u', 0),
 ('cid_g', 0),
 ('cid_b', 0),
 ('cid_r', 1),
 ('mc_c', 2),
 ('mc_w', 0),
 ('mc_u', 0),
 ('mc_b', 0),
 ('mc_r', 0),
 ('mc_g', 2),
 ('power', 6),
 ('toughness', 4),
 ('flying', 0),
 ('firststrike', 0),
 ('doublestrike', 0),
 ('trample', 0),
 ('vigilance', 0),
 ('defender', 0),
 ('haste', 0),
 ('menace', 0),
 ('fear', 0),
 ('deathtouch', 0),
 ('indestructible', 0),
 ('hexproof/shroud', 0)]